In [1]:
from pyFIT3D.common.tools import get_wave_from_header
from pyFIT3D.common.constants import __c__
import warnings
from scipy.optimize import least_squares
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy import units as u
import os
import socket
from tqdm import tqdm
from copy import deepcopy as copy
import itertools as it
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from astropy.visualization import hist
from scipy.stats import gaussian_kde
from scipy.ndimage import generic_filter, gaussian_filter
import pandas as pd
import seaborn as sns
from PIL import Image, ImageEnhance
from lvmdap.analysis import img_scale



sns.set_theme(context="poster", style="ticks",
              palette="Set2", color_codes=True)


warnings.filterwarnings('ignore', category=UserWarning)

In [2]:
from pyFIT3D.common.io import get_wave_from_header
from pyFIT3D.common.tools import cont_seg_all_SN


fits_path = "/disk-a/mejia/Research/UNAM/lvm-dap/_fitting-data/LMC-MUSE/raws"
target_cube = "ADP.2021-04-28T07:59:02.241.fits"

raw = fits.open(f"{fits_path}/{target_cube}")
raw.info()


Filename: /disk-a/mejia/Research/UNAM/lvm-dap/_fitting-data/LMC-MUSE/raws/ADP.2021-04-28T07:59:02.241.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU    2216   ()      
  1  DATA          1 ImageHDU        44   (385, 382, 3801)   float32   
  2  STAT          1 ImageHDU        44   (385, 382, 3801)   float32   


In [3]:
raw[1].header
wave = raw[1].header["CRVAL3"] + raw[1].header["CD3_3"] * \
    (np.arange(raw[1].header["NAXIS3"]) + 1 - raw[1].header["CRPIX3"])
# wave = get_wave_from_header(raw[1].header, wave_axis=3)
# wave

In [ ]:
# scripts:
# 1. download cubes
#   - taken from ESO website
# 2. generate V-band FITS images with WCS
#   - inputs:
#       * raw cube
#   - outputs:
#       * FITS file containing maps and WCS
#   - steps:
#       * calculate median flux within V-band
#       * calculate propagated error within V-band
#       * calculate SNR within V-band
#       * store each map in a separated HDU
#       * calculate WCS
#       * store WCS in first HDU
#       * save FITS file
# 3. coadd cubes of the same pointing
#   - inputs:
#       * list of maps
#   - outputs:
#       * coadded cubes
#   - steps:
#       * read maps
#       * store primary headers
#       * calculate coadded mosaic
#       * display coadded mosaic
#       * identify cubes to coadd
#       * device strategy for coadding cubes (e.g., slice-by-slice)
#       * coadd cubes and propagate errors
#       * save coadded cubes
# 4. preprocess cubes
#   - inputs:
#       * coadded cube
#   - outputs:
#       * RSS fluxes
#       * RSS errors
#       * segmentation map
#       * position table
#   - steps:
#       * read cube
#       * calculate continuum segmented map
#       * save resulting RSS fluxes in FITS file
#       * save resulting RSS errors in FITS file
#       * save position table
